In [4]:
# Loading Libraries

import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

In [2]:
data = pd.read_csv(r"E:\AI\LibrariesForAI\Supervised_Learning\9_Logistic_Regression_multiclass\breast_cancer_data.csv")
data.drop(['id','Unnamed: 32'],axis=1,inplace=True)
# map function for mapping benign into 0 and malignant into 1
data['diagnosis'] = data['diagnosis'].map({'B':0,'M':1})

X=data.drop(['diagnosis'],axis=1)
y=data['diagnosis']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33,random_state=42)

In [6]:
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

In [12]:
# Random Forest
rf_model = RandomForestClassifier()
rf_model.fit(X_train,y_train)
ytrain_pred=rf_model.predict_proba(X_train)

print(f"RF training ROC- AUC Accuarcy is : {roc_auc_score(y_train,ytrain_pred[:,1])}")
ytest_prob = rf_model.predict_proba(X_test)
print(f"RF test ROC- AUC Accuarcy is : {roc_auc_score(y_test,ytest_prob[:,1])}")

RF training ROC- AUC Accuarcy is : 1.0
RF test ROC- AUC Accuarcy is : 0.9956827433082521


In [13]:
# Logistic Regression 
log_model = LogisticRegression()
log_model.fit(X_train,y_train)
ytrain_pred=log_model.predict_proba(X_train)

print(f"Logistic Reg training ROC- AUC Accuarcy is : {roc_auc_score(y_train,ytrain_pred[:,1])}")
ytest_prob = log_model.predict_proba(X_test)
print(f"Logistic Reg  test ROC- AUC Accuarcy is : {roc_auc_score(y_test,ytest_prob[:,1])}")

Logistic Reg training ROC- AUC Accuarcy is : 0.9931911163062537
Logistic Reg  test ROC- AUC Accuarcy is : 0.9975329961761441


c:\Users\dell pc\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [14]:
# Adaboost Classifier
#  
ada_model = AdaBoostClassifier()
ada_model.fit(X_train,y_train)
ytrain_pred=ada_model.predict_proba(X_train)

print(f"Adaboost training ROC- AUC Accuarcy is : {roc_auc_score(y_train,ytrain_pred[:,1])}")
ytest_prob = ada_model.predict_proba(X_test)
print(f"Adaboost   test ROC- AUC Accuarcy is : {roc_auc_score(y_test,ytest_prob[:,1])}")

Adaboost training ROC- AUC Accuarcy is : 1.0
Adaboost   test ROC- AUC Accuarcy is : 0.9861847785864069


In [23]:
# KNN Classifier
#  
knn_model = KNeighborsClassifier()
knn_model.fit(X_train,y_train)
ytrain_pred=knn_model.predict_proba(X_train)

print(f"Adaboost training ROC- AUC Accuarcy is : {roc_auc_score(y_train,ytrain_pred[:,1])}")
ytest_prob = knn_model.predict_proba(X_test)
print(f"Adaboost   test ROC- AUC Accuarcy is : {roc_auc_score(y_test,ytest_prob[:,1])}")

Adaboost training ROC- AUC Accuarcy is : 0.9851548801870251
Adaboost   test ROC- AUC Accuarcy is : 0.9929690391020106


In [47]:
pred =[]
for model in [rf_model,log_model,ada_model,knn_model]:
   pred.append(pd.Series(model.predict_proba(X_test)[:,1]))

final_prediction = pd.concat(pred,axis=1).mean(axis=1)

In [48]:
final_prediction


0      0.255337
1      0.956190
2      0.916487
3      0.096471
4      0.051968
         ...   
183    0.887628
184    0.943512
185    0.831874
186    0.557179
187    0.097301
Length: 188, dtype: float64

In [49]:
# calculate the ROC curve 
fpr, tpr, thresholds = roc_curve(y_test,final_prediction)

In [53]:
thresholds

array([1.96164804, 0.96164804, 0.60505237, 0.55717902, 0.51913308,
       0.51508798, 0.48473507, 0.33019899, 0.29866769, 0.05196831])

In [59]:
final_prediction

0      0.255337
1      0.956190
2      0.916487
3      0.096471
4      0.051968
         ...   
183    0.887628
184    0.943512
185    0.831874
186    0.557179
187    0.097301
Length: 188, dtype: float64

In [64]:
thresholds


array([1.96164804, 0.96164804, 0.60505237, 0.55717902, 0.51913308,
       0.51508798, 0.48473507, 0.33019899, 0.29866769, 0.05196831])

In [57]:
accuracy_ls=[]

for thres in thresholds:
    y_pred = np.where(final_prediction>thres,1,0)
    accuracy_ls.append(accuracy_score(y_test,y_pred))

In [67]:
acc_list = pd.concat([pd.Series(thresholds),pd.Series(accuracy_ls)],axis=1)
acc_list.columns =['Threshold','Accuarcy']
acc_list.sort_values(by='Accuarcy',ascending=False,inplace=True)


In [68]:
acc_list

,Threshold,Accuarcy
3,0.557179,0.984043
5,0.515088,0.984043
2,0.605052,0.978723
4,0.519133,0.978723
6,0.484735,0.978723
7,0.330199,0.952128
8,0.298668,0.946809
0,1.961648,0.643617
1,0.961648,0.643617
9,0.051968,0.361702
